## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-03-22-49-18 +0000,nypost,NY AG blasts tony Southampton’s pot laws after...,https://nypost.com/2025/12/03/us-news/ny-ag-bl...
1,2025-12-03-22-47-37 +0000,nyt,"In Yemen, Separatist Forces Backed by UAE Marc...",https://www.nytimes.com/2025/12/03/world/middl...
2,2025-12-03-22-44-12 +0000,bbc,Watch: What Tennessee's special election means...,https://www.bbc.com/news/videos/c8dy1egqzv1o?a...
3,2025-12-03-22-44-00 +0000,wsj,Opinion | Dr. Prasad’s Children’s Vaccine Crusade,https://www.wsj.com/opinion/vinay-prasad-child...
4,2025-12-03-22-43-22 +0000,cbc,Prime Minister Carney hosts Canadian tennis st...,https://www.cbc.ca/sports/tennis/pm-mark-carne...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,60
93,new,19
113,ukraine,14
560,plan,13
199,he,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
380,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...,176
348,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...,128
48,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...,112
354,2025-12-03-01-41-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,111
342,2025-12-03-02-19-03 +0000,cbc,Trump says National Guard will soon go to New ...,https://www.cbc.ca/news/world/trump-new-orlean...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
380,176,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...
348,68,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...
378,60,2025-12-02-23-45-31 +0000,latimes,"Baby, baby, baby, baby, oh, baby: New gorilla ...",https://www.latimes.com/california/story/2025-...
327,54,2025-12-03-04-10-00 +0000,wsj,Republican Matt Van Epps won the special elect...,https://www.wsj.com/politics/elections/republi...
234,54,2025-12-03-13-33-53 +0000,nypost,Arrogant Putin sees ‘no point in making any se...,https://nypost.com/2025/12/03/world-news/putin...
274,44,2025-12-03-11-00-00 +0000,nypost,Sen. Ron Johnson tells ‘Pod Force One’ he was ...,https://nypost.com/2025/12/03/us-news/sen-ron-...
336,42,2025-12-03-03-10-40 +0000,bbc,YouTube says it will be less safe for kids und...,https://www.bbc.com/news/articles/cp341gz24p5o...
186,40,2025-12-03-16-39-00 +0000,wsj,EU Launches $3.5 Billion Plan to Secure Raw-Ma...,https://www.wsj.com/world/europe/eu-launches-3...
157,39,2025-12-03-18-05-36 +0000,nypost,Private employers cut 32K jobs last month — hi...,https://nypost.com/2025/12/03/business/private...
150,39,2025-12-03-18-28-12 +0000,latimes,Matthew Perry's 'Dr. P' sentenced to prison fo...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
